# Importing the libraries

In [ ]:
import sys
!{sys.executable} -m pip install pandas matplotlib seaborn calmap numpy scipy geopandas shapely folium

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import calmap
import numpy as np
import matplotlib.cm as cm
import scipy.stats as stats
import geopandas as gpd
from shapely.geometry import Point
import folium
from folium.plugins import HeatMap

# Combining and cleaning the dataset

In [2]:
# Set pandas display options to show all columns for .head command
pd.set_option('display.max_columns', None)  
pd.set_option('display.width', None)       
pd.set_option('display.max_colwidth', None) 

In [3]:
# Loading the datasets
file_path1 = r'data\Police_Department_Incident_Reports__Historical_2003_to_May_2018_20250204.csv'
file_path2 = r'data\Police_Department_Incident_Reports__2018_to_Present_20250204.csv'
df1 = pd.read_csv(file_path1)
df2 = pd.read_csv(file_path2)

In [4]:
df1.head()

,PdId,IncidntNum,Incident Code,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,location,SF Find Neighborhoods 2 2,Current Police Districts 2 2,Current Supervisor Districts 2 2,Analysis Neighborhoods 2 2,DELETE - Fire Prevention Districts 2 2,DELETE - Police Districts 2 2,DELETE - Supervisor Districts 2 2,DELETE - Zip Codes 2 2,DELETE - Neighborhoods 2 2,DELETE - 2017 Fix It Zones 2 2,Civic Center Harm Reduction Project Boundary 2 2,Fix It Zones as of 2017-11-06 2 2,DELETE - HSOC Zones 2 2,Fix It Zones as of 2018-02-07 2 2,"CBD, BID and GBD Boundaries as of 2017 2 2","Areas of Vulnerability, 2016 2 2",Central Market/Tenderloin Boundary 2 2,Central Market/Tenderloin Boundary Polygon - Updated 2 2,HSOC Zones as of 2018-06-05 2 2,OWED Public Spaces 2 2,Neighborhoods 2
0,4133422003074,41334220,3074,ROBBERY,"ROBBERY, BODILY FORCE",Monday,11/22/2004,17:50,INGLESIDE,NONE,GENEVA AV / SANTOS ST,-122.420084,37.708311,POINT (-122.420084075249 37.7083109744362),NaN,NaN,9.0,NaN,NaN,NaN,8.0,309.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5118535807021,51185358,7021,VEHICLE THEFT,STOLEN AUTOMOBILE,Tuesday,10/18/2005,20:00,PARK,NONE,TURK ST / STJOSEPHS AV,-120.500000,90.000000,POINT (-120.50000000000001 90),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4018830907021,40188309,7021,VEHICLE THEFT,STOLEN AUTOMOBILE,Sunday,02/15/2004,02:00,SOUTHERN,NONE,BRANNAN ST / 1ST ST,-120.500000,90.000000,POINT (-120.50000000000001 90),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11014543126030,110145431,26030,ARSON,ARSON,Friday,02/18/2011,05:27,INGLESIDE,NONE,0 Block of SANJUAN AV,-122.436220,37.724377,POINT (-122.43622001281001 37.7243766140428),94.0,9.0,1.0,28.0,9.0,7.0,6.0,28861.0,25.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,94.0
4,10108108004134,101081080,4134,ASSAULT,BATTERY,Sunday,11/21/2010,17:00,SOUTHERN,NONE,400 Block of 10TH ST,-122.410541,37.770913,POINT (-122.410541166987 37.7709130566165),32.0,1.0,10.0,34.0,8.0,2.0,9.0,28853.0,34.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,32.0


In [5]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2129525 entries, 0 to 2129524
Data columns (total 35 columns):
 #   Column                                                    Dtype  
---  ------                                                    -----  
 0   PdId                                                      int64  
 1   IncidntNum                                                int64  
 2   Incident Code                                             int64  
 3   Category                                                  object 
 4   Descript                                                  object 
 5   DayOfWeek                                                 object 
 6   Date                                                      object 
 7   Time                                                      object 
 8   PdDistrict                                                object 
 9   Resolution                                                object 
 10  Address                       

In [6]:
# Select and rename columns for df1 (2003-2018)
df1 = df1.rename(columns={
    'IncidntNum': 'incident_number',
    'Category': 'category',
    'Date': 'date',
    'Time': 'time',
    'DayOfWeek': 'day_of_week',
    'PdDistrict': 'police_district',
    'Resolution': 'resolution',
    'X': 'longitude',
    'Y': 'latitude'
})[[
    'incident_number', 'category', 'date', 'time', 'day_of_week', 
    'police_district', 'resolution', 'longitude', 'latitude'
]]

# Select and rename columns for df2 (2018-present)
df2 = df2.rename(columns={
    'Incident Number': 'incident_number',
    'Incident Category': 'category',
    'Incident Date': 'date',
    'Incident Time': 'time',
    'Incident Day of Week': 'day_of_week',
    'Police District': 'police_district',
    'Resolution': 'resolution',
    'Longitude': 'longitude',
    'Latitude': 'latitude'
})[[
    'incident_number', 'category', 'date', 'time', 'day_of_week', 
    'police_district', 'resolution', 'longitude', 'latitude'
]]

In [7]:
# Convert date column to datetime format
df1['date'] = pd.to_datetime(df1['date'])
df2['date'] = pd.to_datetime(df2['date'])

In [8]:
# Find the range of years in df2
full_years = df2['date'].dt.year.value_counts().sort_index()
print("Crime records per year in df2:\n", full_years)

# Find the last full year
latest_year = full_years.index.max()
earliest_year = full_years.index.min()

# Check if the latest year has data for all 12 months
if df2[df2['date'].dt.year == latest_year]['date'].dt.month.nunique() < 12:
    print(f"Year {latest_year} is incomplete. Removing it.")
    df2 = df2[df2['date'].dt.year < latest_year] 

Crime records per year in df2:
 date
2018    151581
2019    146714
2020    117503
2021    128348
2022    135658
2023    133072
2024    108307
2025      7572
Name: count, dtype: int64
Year 2025 is incomplete. Removing it.


In [9]:
# Apply filtering to keep full years only
df1 = df1[(df1['date'].dt.year >= 2003) & (df1['date'].dt.year <= 2017)]
df2 = df2[df2['date'].dt.year >= earliest_year]  

In [10]:
# Merge both datasets
df = pd.concat([df1, df2], ignore_index=True)

In [11]:
# list the column names for df
print("df column list:")   
print(df.columns)

df column list:
Index(['incident_number', 'category', 'date', 'time', 'day_of_week',
       'police_district', 'resolution', 'longitude', 'latitude'],
      dtype='object')


In [12]:
# check for missing values
print("Missing values per column:")
print(df.isnull().sum())

Missing values per column:
incident_number        0
category            1005
date                   0
time                   0
day_of_week            0
police_district        1
resolution             0
longitude          50598
latitude           50598
dtype: int64


In [13]:
# drop rows with missing values
df = df.dropna()

In [14]:
# check for nan values
print("Missing values per column after dropping rows with missing values:")
print(df.isnull().sum())

Missing values per column after dropping rows with missing values:
incident_number    0
category           0
date               0
time               0
day_of_week        0
police_district    0
resolution         0
longitude          0
latitude           0
dtype: int64


In [15]:
# describe the data
print("Data description:")
print(df.describe(include='all'))


Data description:
        incident_number       category                           date  \
count      2.954060e+06        2954060                        2954060   
unique              NaN             86                            NaN   
top                 NaN  LARCENY/THEFT                            NaN   
freq                NaN         464364                            NaN   
mean       1.341681e+08            NaN  2013-10-08 04:06:56.216867328   
min        0.000000e+00            NaN            2003-01-01 00:00:00   
25%        8.061348e+07            NaN            2008-05-23 00:00:00   
50%        1.360906e+08            NaN            2013-11-18 00:00:00   
75%        1.861793e+08            NaN            2018-11-28 00:00:00   
max        9.915824e+08            NaN            2024-12-31 00:00:00   
std        6.291862e+07            NaN                            NaN   

           time day_of_week police_district resolution     longitude  \
count   2954060     2954060      

In [16]:
# print unique values in the category column and total number of unique values
print("Unique values in the category column:")
print(df['category'].unique())
print("Total number of unique values:", df['category'].nunique())

Unique values in the category column:
['ROBBERY' 'VEHICLE THEFT' 'ARSON' 'ASSAULT' 'TRESPASS' 'BURGLARY'
 'LARCENY/THEFT' 'WARRANTS' 'OTHER OFFENSES' 'DRUG/NARCOTIC'
 'SUSPICIOUS OCC' 'LIQUOR LAWS' 'VANDALISM' 'WEAPON LAWS' 'NON-CRIMINAL'
 'MISSING PERSON' 'FRAUD' 'SEX OFFENSES, FORCIBLE' 'SECONDARY CODES'
 'DISORDERLY CONDUCT' 'RECOVERED VEHICLE' 'KIDNAPPING'
 'FORGERY/COUNTERFEITING' 'PROSTITUTION' 'DRUNKENNESS' 'BAD CHECKS'
 'DRIVING UNDER THE INFLUENCE' 'LOITERING' 'STOLEN PROPERTY' 'SUICIDE'
 'BRIBERY' 'EXTORTION' 'EMBEZZLEMENT' 'GAMBLING' 'PORNOGRAPHY/OBSCENE MAT'
 'SEX OFFENSES, NON FORCIBLE' 'TREA' 'Lost Property' 'Assault'
 'Non-Criminal' 'Other Miscellaneous' 'Other' 'Suspicious Occ'
 'Disorderly Conduct' 'Rape' 'Missing Person' 'Fire Report' 'Arson'
 'Larceny Theft' 'Malicious Mischief' 'Fraud' 'Burglary' 'Warrant'
 'Robbery' 'Vandalism' 'Other Offenses' 'Suicide' 'Traffic Collision'
 'Courtesy Report' 'Drug Offense' 'Motor Vehicle Theft'
 'Traffic Violation Arrest' 'Miscell

In [17]:
category_mapping = {
    # Focus crimes
    'Robbery': 'ROBBERY',
    'Burglary': 'BURGLARY',
    'Assault': 'ASSAULT',
    'Drug Violation': 'DRUG/NARCOTIC',
    'Drug Offense': 'DRUG/NARCOTIC',
    'Larceny Theft': 'LARCENY/THEFT',
    'Vandalism': 'VANDALISM',
    'Malicious Mischief': 'VANDALISM',  
    'Motor Vehicle Theft': 'VEHICLE THEFT',
    'Motor Vehicle Theft?': 'VEHICLE THEFT',
    'Stolen Property': 'STOLEN PROPERTY',
    'Disorderly Conduct': 'DISORDERLY CONDUCT',
    'Weapons Carrying Etc': 'WEAPON LAWS',
    'Weapons Offense': 'WEAPON LAWS',
    'Weapons Offence': 'WEAPON LAWS',
    'Prostitution': 'PROSTITUTION',
    # Additional mappings for edge cases
    'Traffic Violation Arrest': 'DRIVING UNDER THE INFLUENCE',  
    'Suspicious Occ': 'TRESPASS',
    'Suspicious': 'TRESPASS',
    'Liquor Laws': 'DRUNKENNESS'  
}

# Apply the mapping to the category column
df['category'] = df['category'].replace(category_mapping)

In [18]:
# Check the unique values in the category column again
print("Unique values in the category column after mapping:")
print(df['category'].unique())
print("Total number of unique values:", df['category'].nunique())

Unique values in the category column after mapping:
['ROBBERY' 'VEHICLE THEFT' 'ARSON' 'ASSAULT' 'TRESPASS' 'BURGLARY'
 'LARCENY/THEFT' 'WARRANTS' 'OTHER OFFENSES' 'DRUG/NARCOTIC'
 'SUSPICIOUS OCC' 'LIQUOR LAWS' 'VANDALISM' 'WEAPON LAWS' 'NON-CRIMINAL'
 'MISSING PERSON' 'FRAUD' 'SEX OFFENSES, FORCIBLE' 'SECONDARY CODES'
 'DISORDERLY CONDUCT' 'RECOVERED VEHICLE' 'KIDNAPPING'
 'FORGERY/COUNTERFEITING' 'PROSTITUTION' 'DRUNKENNESS' 'BAD CHECKS'
 'DRIVING UNDER THE INFLUENCE' 'LOITERING' 'STOLEN PROPERTY' 'SUICIDE'
 'BRIBERY' 'EXTORTION' 'EMBEZZLEMENT' 'GAMBLING' 'PORNOGRAPHY/OBSCENE MAT'
 'SEX OFFENSES, NON FORCIBLE' 'TREA' 'Lost Property' 'Non-Criminal'
 'Other Miscellaneous' 'Other' 'Rape' 'Missing Person' 'Fire Report'
 'Arson' 'Fraud' 'Warrant' 'Other Offenses' 'Suicide' 'Traffic Collision'
 'Courtesy Report' 'Miscellaneous Investigation' 'Case Closure'
 'Sex Offense' 'Recovered Vehicle' 'Forgery And Counterfeiting'
 'Vehicle Misplaced' 'Offences Against The Family And Children'
 'Vehi

In [19]:
# setting up focus crimes
focus_crimes = [
    'WEAPON LAWS', 'PROSTITUTION', 'ROBBERY', 'BURGLARY', 'ASSAULT', 'DRUG/NARCOTIC', 'LARCENY/THEFT', 'VANDALISM', 'VEHICLE THEFT', 'STOLEN PROPERTY']

# Filter the data for focus crimes only
df_focus = df[df['category'].isin(focus_crimes)]

In [20]:
df.head()

,incident_number,category,date,time,day_of_week,police_district,resolution,longitude,latitude
0,41334220,ROBBERY,2004-11-22,17:50,Monday,INGLESIDE,NONE,-122.420084,37.708311
1,51185358,VEHICLE THEFT,2005-10-18,20:00,Tuesday,PARK,NONE,-120.500000,90.000000
2,40188309,VEHICLE THEFT,2004-02-15,02:00,Sunday,SOUTHERN,NONE,-120.500000,90.000000
3,110145431,ARSON,2011-02-18,05:27,Friday,INGLESIDE,NONE,-122.436220,37.724377
4,101081080,ASSAULT,2010-11-21,17:00,Sunday,SOUTHERN,NONE,-122.410541,37.770913


In [21]:
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month_name() 
df['hour'] = df['time'].str.split(':').str[0].astype(int)
df['day of the month'] = df['date'].dt.day


In [22]:
df.head()

,incident_number,category,date,time,day_of_week,police_district,resolution,longitude,latitude,year,month,hour,day of the month
0,41334220,ROBBERY,2004-11-22,17:50,Monday,INGLESIDE,NONE,-122.420084,37.708311,2004,November,17,22
1,51185358,VEHICLE THEFT,2005-10-18,20:00,Tuesday,PARK,NONE,-120.500000,90.000000,2005,October,20,18
2,40188309,VEHICLE THEFT,2004-02-15,02:00,Sunday,SOUTHERN,NONE,-120.500000,90.000000,2004,February,2,15
3,110145431,ARSON,2011-02-18,05:27,Friday,INGLESIDE,NONE,-122.436220,37.724377,2011,February,5,18
4,101081080,ASSAULT,2010-11-21,17:00,Sunday,SOUTHERN,NONE,-122.410541,37.770913,2010,November,17,21


In [23]:
# drop resolution column    
df = df.drop(columns=['resolution'])
df.head()

,incident_number,category,date,time,day_of_week,police_district,longitude,latitude,year,month,hour,day of the month
0,41334220,ROBBERY,2004-11-22,17:50,Monday,INGLESIDE,-122.420084,37.708311,2004,November,17,22
1,51185358,VEHICLE THEFT,2005-10-18,20:00,Tuesday,PARK,-120.500000,90.000000,2005,October,20,18
2,40188309,VEHICLE THEFT,2004-02-15,02:00,Sunday,SOUTHERN,-120.500000,90.000000,2004,February,2,15
3,110145431,ARSON,2011-02-18,05:27,Friday,INGLESIDE,-122.436220,37.724377,2011,February,5,18
4,101081080,ASSAULT,2010-11-21,17:00,Sunday,SOUTHERN,-122.410541,37.770913,2010,November,17,21


In [24]:
# uniqye values in the police_district column
print("Unique values in the police_district column:")
print(df['police_district'].unique())
print("Total number of unique values:", df['police_district'].nunique())

Unique values in the police_district column:
['INGLESIDE' 'PARK' 'SOUTHERN' 'TARAVAL' 'TENDERLOIN' 'BAYVIEW' 'CENTRAL'
 'NORTHERN' 'MISSION' 'RICHMOND' 'Central' 'Bayview' 'Northern' 'Southern'
 'Mission' 'Richmond' 'Park' 'Tenderloin' 'Ingleside' 'Taraval'
 'Out of SF']
Total number of unique values: 21


In [25]:
df = df[df['police_district'] != 'Out of SF'] # this value is only available in 2nd dataset so we remove it

In [26]:
print("Unique values in the police_district column:")
print(df['police_district'].unique())
print("Total number of unique values:", df['police_district'].nunique())

Unique values in the police_district column:
['INGLESIDE' 'PARK' 'SOUTHERN' 'TARAVAL' 'TENDERLOIN' 'BAYVIEW' 'CENTRAL'
 'NORTHERN' 'MISSION' 'RICHMOND' 'Central' 'Bayview' 'Northern' 'Southern'
 'Mission' 'Richmond' 'Park' 'Tenderloin' 'Ingleside' 'Taraval']
Total number of unique values: 20


In [27]:
pd_mapping = {
    # PdDistrict text values should match in both datasets
    'Bayview': 'BAYVIEW',
    'Mission': 'MISSION',
    'Northern': 'NORTHERN',
    'Central': 'CENTRAL',
    'Richmond': 'RICHMOND',
    'Ingleside': 'INGLESIDE',
    'Southern': 'SOUTHERN',
    'Park': 'PARK',  
    'Taraval': 'TARAVAL',
    'Tenderloin': 'TENDERLOIN',
}

# Apply the mapping to the police_district column
df['police_district'] = df['police_district'].replace(pd_mapping)

In [28]:
print("Unique values in the police_district column:")
print(df['police_district'].unique())
print("Total number of unique values:", df['police_district'].nunique())

Unique values in the police_district column:
['INGLESIDE' 'PARK' 'SOUTHERN' 'TARAVAL' 'TENDERLOIN' 'BAYVIEW' 'CENTRAL'
 'NORTHERN' 'MISSION' 'RICHMOND']
Total number of unique values: 10


In [ ]:
# save the cleaned dataset sf as a csv file
df.to_csv('sf_cleaned.csv', index=False)